In [5]:
import pandas as pd
import numpy as np


####solver
from scipy.optimize import fsolve


import matplotlib.pyplot as plt
%matplotlib inline


## Short Interest Tree Model

### Example from book "Fixed Income Securities by Pietro Veronesi

I am using this book as a reference to create my functions then to test them. 
At a later stage, I will try to calibrate "theta".

In [6]:
def c2c(r, n):
    """
    Continuous to compounded rate

    Args:
        r (float): continuous interest rate
        n (int): number of compounding

    Returns:
        float: compounded interest rate
    """
    return n * (np.exp(r/n)-1)

## Ho-Lee Model

In [46]:
class HoLee(object):
    
    '''
    Class to calibrate Binamial Tree to fit the interest rate
    using Ho Lee implementation
    ============================
    n: number of time period
    T: number of years. 
    dt: T/n
    zcb: array, price of zero coupon bonds. 
        Make sure these are ZCB. Only check is if zcb >1, then devide by 100. 
        
    sigma = Annualised Volatility (standard deviation)
    
    =============================
    Assumption: dt is constant
    Future development ideas: 
        If dt between each price not constant, 
        implement raw interpolation  
    
    '''
    
    def __init__(self, zcb, T,  sigma ):
        
        self.zcb = np.array(zcb) # Array
        self.n = len(self.zcb) 
        self.T = T
        self.sigma = sigma # Annualised volatility 
        self.dt = T/self.n
        
        self.rates = np.zeros((self.n, self.n))
        self.thetas = np.nan # store theta's value once calibrated
        
        # if ZCB > 1 ==> ZCB quoted per $100. 
        if self.zcb[-1] > 1.0:
            self.zcb /= 100
        
        # Extract first interest rate (Trivial)
        self.rates[0,0] = -np.log(self.zcb[0])/self.dt
        
        # Ideas, if dt not fixed, insert array along zcb to have time. 


        
    @staticmethod
    def forward_tree(r0,sigma,dt,thetas):
        """
        Forward tree valuation
        Works for both Ho Lee and BDT model (change r_0 to ln r_0)

        Args:
            r0 (float): _description_
            sigma (float): _description_
            dt (float): _description_
            thetas (array / list): _description_

        Returns:
             nxn array: interest rates
             
        =========================
        Uses backward tree for Bond evaluation
        Future area of improvement, maybe use
        backward_tree method
        """
        n = len(thetas)
        tree_rate = np.zeros((n+1,n+1))
        tree_rate[0,0] = r0
        tree_zcb = np.zeros((n+2, n+2))
        tree_zcb[:, -1] = 1.0 # Could be 100.0

        for i in range(n):
            tree_rate[0,i+1] = tree_rate[0,i] + thetas[i] * dt \
                + sigma * np.sqrt(dt)
            
            
            # Vectorise calculation by -2 sigma on each row
            # (column-wise operation)
            
            # ignore first row (already calculated)
            tree_rate[1:i+2,i+1] = tree_rate[0,i+1] \
                - 2 * np.arange(1,i+2) * sigma * np.sqrt(dt)
            
        # Calculate ZCB backward
        

        for i in np.arange(n, -1, -1):

            
            tree_zcb[0:i+1,i] = np.exp(-tree_rate[:i+1,i] * dt) \
                * 0.5 * (tree_zcb[0:i+1,i+1] + tree_zcb[1:i+2,i+1])
            

        return tree_rate, tree_zcb
            
    def fit_theta(self):
        """
        Find theta parameters
        -------------------------
        In financial mathematics, the Ho–Lee model is a short rate model widely used 
        in the pricing of bond options, swaptions and other interest rate derivatives,
        and in modeling future interest rates. It was developed in 1986 by Thomas Ho and Sang Bin Lee.
        (from Wikepedia: https://en.wikipedia.org/wiki/Ho%E2%80%93Lee_model)
        
        - Risk Neutral probability = 0.5
        
        Drawback: Ho-Lee model allows negative interest rate
        
        Great ressource: 
        https://www.bensblog.tech/fixed_income/HoLee_Model/
        """
        thetas=[]

        r0 = self.rates[0,0]
        
        
        for i in self.zcb[1:]:
            p0=i
            func = (lambda t: self.forward_tree(r0,self.sigma,self.dt,thetas+[t])[1][0,0]-p0)
            new_theta=fsolve(func,0.001)
            thetas.append(new_theta[0])
            
        self.thetas = thetas
        self.rates = self.forward_tree(r0,self.sigma,self.dt,thetas)[0]


## Black Derman Toy

In mathematical finance, the Black–Derman–Toy model (BDT) is a popular short rate model
        used in the pricing of bond options, swaptions and other interest rate derivatives.
        
        Wikipedia: 
        https://en.wikipedia.org/wiki/Black%E2%80%93Derman%E2%80%93Toy_model
        

In [47]:
class BlackDermanToy(object):
    
    '''
    Class to calibrate Binamial Tree to fit the interest rate
    using  Black Derman Toy implementation
    ============================
    n: number of time period
    T: number of years. 
    dt: T/n
    zcb: array, price of zero coupon bonds. 
       
    sigma = vol of log interest rate!(standard deviation)
    
    =============================
    Assumption: dt is constant
    Future development ideas: 
        If dt between each price not constant, 
        implement raw interpolation  
    
    =============================

    '''
    
    def __init__(self, zcb, T,  sigma ):
        
        self.zcb = np.array(zcb) # Array
        self.n = len(self.zcb) 
        self.T = T
        self.sigma = sigma # vol of log interest rate!
        self.dt = T/self.n
        
        self.rates = np.zeros((self.n, self.n))
        self.thetas = np.nan # store theta's value once calibrated
        
        # if ZCB > 1 ==> ZCB quoted per $100. 
        if self.zcb[-1] > 1.0:
            self.zcb /= 100
        
        # Extract first interest rate (Trivial)
        self.rates[0,0] = -np.log(self.zcb[0])/self.dt
        
        # Ideas, if dt not fixed, insert array along zcb to have time. 


        
    @staticmethod
    def forward_tree(r0,sigma,dt,thetas):
        """
        Forward tree valuation
        Works for both Ho Lee and BDT model (change r_0 to ln r_0)

        Args:
            r0 (float): _description_
            sigma (float): _description_
            dt (float): _description_
            thetas (array / list): _description_

        Returns:
             nxn array: interest rates
             
        =========================
        Uses backward tree for Bond evaluation
        Future area of improvement, maybe use
        backward_tree method
        """
        n = len(thetas)
        tree_rate = np.zeros((n+1,n+1))
        tree_rate[0,0] = np.log(r0)
        tree_zcb = np.zeros((n+2, n+2))
        tree_zcb[:, -1] = 1.0 # Could be 100.0

        for i in range(n):
            tree_rate[0,i+1] = tree_rate[0,i] + thetas[i] * dt \
                + sigma * np.sqrt(dt)
            
            
            # Vectorise calculation by -2 sigma on each row
            # (column-wise operation)
            
            # ignore first row (already calculated)
            tree_rate[1:i+2,i+1] = tree_rate[0,i+1] \
                - 2 * np.arange(1,i+2) * sigma * np.sqrt(dt)
            
        # Calculate ZCB backward
        

        for i in np.arange(n, -1, -1):

            r = np.exp(tree_rate[:i+1,i])
            tree_zcb[0:i+1,i] = np.exp(-r * dt) \
                * 0.5 * (tree_zcb[0:i+1,i+1] + tree_zcb[1:i+2,i+1])
            
        
        
        
        # z_i = ln(r_i) <=> r_i = exp(z_i)
        # replace 1.0 by 0.0 
        tree_rate = np.exp(tree_rate)
        for i in range(len(tree_rate)):
            tree_rate[i+1:,i] = 0
       
        return tree_rate, tree_zcb
            
    def fit_theta(self):
        """
        Find theta parameters
        -------------------------
        
        ----------------------------------
        Note that differently form the Ho-Lee model, now sigma is the
        vol of log-interest rates. 
        - z_i = log(r_i) 
        - Risk Neutral probability = 0.5
        ----------------------------------
        Drawback: 
         - No analytical solution
        
        Great ressource: 
        https://www.bensblog.tech/fixed_income/HoLee_Model/
        """
        thetas=[]
        r0 = self.rates[0,0]
        
        for i in self.zcb[1:]:
            p0=i
            func = (lambda t: self.forward_tree(r0,self.sigma,self.dt,thetas+[t])[1][0,0]-p0)
            new_theta=fsolve(func,.001)
            thetas.append(new_theta[0])
            
        self.thetas = thetas
        self.rates = self.forward_tree(r0,self.sigma,self.dt,thetas)[0]


### Future ideas:

Input backward tree within the implementation

In [48]:
def backward_tree(rates, dt, last = 100):
    '''
    backward tree
    rates: Binomial Tree of interest rate
            used to price bond.
    last: value at maturity

    '''
    n = len(rates)
    tree = np.zeros((n + 1, n + 1)) # bond price tree
    tree[:,-1] = last

    for i in range(n-1,-1,-1):
        for j in range(n-1,-1,-1):
            if j<=i:
                # if p != 0.5 (as per model), change this line
                tree[j,i] = np.exp(-rates[j,i] * dt) \
                    * 0.5 * (tree[j,i+1] + tree[j+1, i+1])


    return tree

Input a numerical solver method

In [49]:
def secant_method(f, estimate = 0.05):
    """Return the root calculated using the secant method."""
    
    iterations=5e2
    
    p0 = estimate * 1.0
    funcall = 0
    eps =1e-4
    p1 = estimate*(1+eps)
    p1 += (eps if p1 >= 0 else -eps)
    
    funcalls = 0
    
    q0 = f(p0)
    funcalls += 1
    q1 = f(p1)
    funcalls += 1

    if abs(q1) < abs(q0):
        p0, p1, q0, q1 = p1, p0, q1, q0
    for itr in range(int(iterations)):
        if q1 == q0:
            if p1 != p0:
                msg = "Tolerance of %s reached." % (p1 - p0)
                if disp:
                    msg += (
                        " Failed to converge after %d iterations, value is %s."
                        % (itr + 1, p1))
                    raise RuntimeError(msg)
                warnings.warn(msg, RuntimeWarning)
            p = (p1 + p0) / 2.0
    else:
            if abs(q1) > abs(q0):
                p = (-q0 / q1 * p1 + p0) / (1 - q0 / q1)
            else:
                p = (-q1 / q0 * p0 + p1) / (1 - q1 / q0)

    return p
    

## Testing Class

### Data

In [50]:
# Table 10.11 Zero Coupon Bond Prices on January 8, 2002,
# Source The Wall Street Journal
# Taken from Fixed Income Securities - Pietro Veronesi || p.370


maturity = np.arange(0.5,6.0,0.5)

price = [99.1338, 97.8925, 96.1462,
         94.1011, 91.7136, 89.2258,
         86.8142, 84.5016, 82.1848,
         79.7718, 77.4339]

yield_ = [1.74, 2.13, 2.62, 3.04,
          3.46,3.8,4.04,4.21,4.36, 
          4.52,4.65]
sigma_hl = 0.0173
sigma_bdt = 0.2142
df = pd.DataFrame([maturity, price, yield_]).T
df.columns = ["Maturity", "Price", "Yield"]

#### Table: 11.2; p 384

In [51]:
# Theta from book
theta_hl = [0.015674,0.021824,0.014374,0.017324,0.007873,0.000423,-0.000628,0.004322,0.009271,0.001202]
theta_bdt = [0.7182,  0.6916, 0.3348, 0.3379, 0.1182, -0.023, -0.0438, 0.0455, 0.1281, -0.0126]


In [52]:
# Fit Ho - Lee
hl=HoLee(price, 5.5, 0.0173)

hl.fit_theta()


In [53]:
# Fit Black Derman Toy
bdt=BlackDermanToy(price, 5.5, 0.2142)

bdt.fit_theta()

In [54]:
# Ho-Lee
temp = pd.DataFrame([theta_hl, hl.thetas]).T
temp["diff"]= temp.iloc[:,0] - temp.iloc[:,1]
temp.columns = ["Book", "Value", "Diff"]
temp

,Book,Value,Diff
0,0.015674,0.015678,-3.949304e-06
1,0.021824,0.021822,1.646897e-06
2,0.014374,0.014375,-9.834329e-07
3,0.017324,0.017319,4.944986e-06
4,0.007873,0.007879,-5.767581e-06
5,0.000423,0.000421,1.904846e-06
6,-0.000628,-0.000629,1.018752e-06
7,0.004322,0.004323,-1.008647e-06
8,0.009271,0.009272,-6.841671e-07
9,0.001202,0.001200,1.663697e-06


In [55]:
# Black Derman Toy
temp = pd.DataFrame([theta_bdt, bdt.thetas]).T
temp["diff"]= temp.iloc[:,0] - temp.iloc[:,1]
temp.columns = ["Book", "Value", "Diff"]
temp

,Book,Value,Diff
0,0.7182,0.718322,-1.217830e-04
1,0.6916,0.691521,7.901000e-05
2,0.3348,0.334844,-4.433694e-05
3,0.3379,0.337824,7.616040e-05
4,0.1182,0.118293,-9.296763e-05
5,-0.0230,-0.023000,-4.556782e-07
6,-0.0438,-0.043815,1.502993e-05
7,0.0455,0.045526,-2.575932e-05
8,0.1281,0.128076,2.381271e-05
9,-0.0126,-0.012640,3.971751e-05


### Observations:
Thetas are similar!

## Caps and Floor

Caps are like european calls <br>
Floors are like european puts

In [ ]:
class Option_IR(object):
    
    def __init__(self,T, n):
        ''' in case I need to write something'''
        
        self.T = T        
        self.n = n

        self.dt = T/n
       
    
    def ctns_rate(self,r):
        
        return (np.exp(r*self.dt)-1)/self.dt
        
        
        
    def european(self, tree_rate, Strike,N, cp = "cap"):



        p = 0.5
        
        tree_ctns = self.ctns_rate(tree_rate)
        
        
        tree_cf = np.zeros((self.n+1,self.n+1)) # create an empty array for Cash-Flow tree
        
        for i in range(self.n+1):
            for j in range(self.n+1):
                if j>i:
                    tree_cf[j,i] = 0
                else:
                    if cp =='cap':
                        tree_cf[j,i] = self.dt * N * max(tree_ctns[j,i] - Strike,0)
                    else:
                        tree_cf[j,i] = self.dt * N * max(Strike - tree_ctns[j,i],0)
                    
        tree_eu = np.zeros((self.n+1,self.n+1))
        
        
        #intresic value
        for j in range(self.n+1):
            tree_eu[j,-1] = np.exp(-tree_rate[j,-1] * self.dt) * tree_cf[j,-1]
            
            
                

        #Backward tree (European option)
        for i in range(self.n,0,-1):
            for j in range(i):
                tree_eu[j,i-1] = np.exp(-tree_rate[j,i-1] * self.dt)*(( p * tree_eu[j,i] + (1-p) * tree_eu[j+1,i])+tree_cf[j,i-1])

        return tree_cf, tree_eu
    

    
    def swap_rate(self, zcb):
        
        return 1/self.dt * (1 - zcb[-1])/ (sum(zcb))
    

    def swap(self, tree_rate, c, N):
        
        p = 0.5
        
        tree_ctns = self.ctns_rate(tree_rate)
        
        
        tree_cf = np.zeros((self.n+1,self.n+1)) # create an empty array for Cash-Flow tree
        
        for i in range(self.n+1):
            for j in range(self.n+1):
                if j>i:
                    tree_cf[j,i] = 0
                else:
                    tree_cf[j,i] = self.dt * N * (tree_ctns[j,i] - c)
                    
        tree_eu = np.zeros((self.n+1,self.n+1))
        
        
        #intresic value
        for j in range(self.n+1):
            tree_eu[j,-1] = np.exp(-tree_rate[j,-1] * self.dt) * tree_cf[j,-1]
            
            
                

        #Backward tree (European option)
        for i in range(self.n,0,-1):
            for j in range(i):
                tree_eu[j,i-1] = np.exp(-tree_rate[j,i-1] * self.dt)*(( p * tree_eu[j,i] + (1-p) * tree_eu[j+1,i])+tree_cf[j,i-1])

        return tree_cf, tree_eu


In [ ]:
pd.DataFrame(Option_IR(4.5,9).swap(y,c, 100)[1])

,0,1,2,3,4,5,6,7,8,9
0,0.00689,4.272425,8.188949,11.385667,13.866054,15.227770,15.502382,14.717643,12.586277,8.204988
1,0.00000,-1.519939,2.046496,5.049234,7.484485,9.030847,9.717720,9.580177,8.417565,5.582189
2,0.00000,0.000000,-2.785965,0.050314,2.445034,4.146658,5.180691,5.581978,5.209567,3.596844
3,0.00000,0.000000,0.000000,-3.826893,-1.466314,0.361367,1.677267,2.512431,2.767122,2.103602
4,0.00000,0.000000,0.000000,0.000000,-4.463328,-2.535859,-0.996939,0.179442,0.922178,0.985778
5,0.00000,0.000000,0.000000,0.000000,0.000000,-4.732939,-3.020801,-1.580577,-0.463315,0.151905
6,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,-4.542809,-2.901052,-1.499293,-0.468546
7,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.887730,-2.271460,-0.929320
8,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.845639,-1.271028
9,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.524176


##### Assign your tree to a variable x:

In [ ]:
x = Tree(1,2).bdt(0.0174,0.2142,theta1)
pd.DataFrame(x)

,0,1,2
0,0.0174,0.028992,0.047670
1,0.0000,0.021415,0.035211
2,0.0000,0.000000,0.026009


##### Cash Flow tree: (p.390)

In [ ]:
pd.DataFrame(Option_IR(1,2).european(x,0.025,100,'cap')[0]) # Cash flow( Please note it arrives a T+1)

,0,1,2
0,0.0,0.210176,1.162114
1,0.0,0.000000,0.526160
2,0.0,0.000000,0.058947


##### Cap Value Tree: (p.391)

In [ ]:
pd.DataFrame(Option_IR(1,2).european(x,0.025,100,'cap')[1])

,0,1,2
0,0.647167,1.021126,1.134743
1,0.000000,0.284518,0.516978
2,0.000000,0.000000,0.058185


##### Cash Flow, 5-year Cap: Panel A, p 392

In [ ]:
y = Tree(4.5,9).bdt(0.0174,0.2142,theta1)
pd.DataFrame(Option_IR(4.5,9).european(y,0.025,100,'cap')[0])

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.210176,1.162114,2.082966,3.370518,4.483962,5.373335,6.323442,7.828410,10.131965
1,0.0,0.000000,0.526160,1.201337,2.142751,2.954425,3.601152,4.290489,5.379150,7.037926
2,0.0,0.000000,0.058947,0.554951,1.245117,1.838851,2.311063,2.813549,3.605376,4.807823
3,0.0,0.000000,0.000000,0.080115,0.587084,1.022503,1.368336,1.735888,2.314145,3.190092
4,0.0,0.000000,0.000000,0.000000,0.103738,0.423658,0.677501,0.947041,1.370591,2.011013
5,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.170201,0.368239,0.679156,1.148637
6,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.171417,0.516267
7,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.051672
8,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


##### Cash Flow, 5-year Cap: Panel B, p 392

In [ ]:
pd.DataFrame(Option_IR(4.5,9).european(y,0.025,100,'cap')[1])

,0,1,2,3,4,5,6,7,8,9
0,9.439097,12.18811,15.098181,17.348980,18.926699,19.437904,18.889858,17.286189,14.325789,9.096594
1,0.000000,6.85504,9.213622,11.251525,12.761801,13.423482,13.247149,12.249134,10.218022,6.499271
2,0.000000,0.00000,4.644049,6.450698,7.890201,8.680570,8.819725,8.328137,7.056592,4.533210
3,0.000000,0.00000,0.000000,2.841083,4.134157,5.003337,5.399971,5.317338,4.649420,3.054471
4,0.000000,0.00000,0.000000,0.000000,1.463359,2.242706,2.789097,3.028711,2.831020,1.947505
5,0.000000,0.00000,0.000000,0.000000,0.000000,0.516157,0.924069,1.302001,1.465404,1.121731
6,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.120979,0.231983,0.444260,0.507306
7,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012510,0.025240,0.051008
8,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


###### Zero Coupon Bond Price on January 8, 2002 (p 370)

In [ ]:
zcb_08 = [99.1338, 97.8925,
      96.462, 94.1011,
      91.7136, 89.2258,
      86.8142, 84.5016,
      82.1848, 79.7718, 77.4339]

In [ ]:
del zcb_08[-1] # remove the last element of the list
zcb = [i/100 for i in zcb_08] # divide each element by 100
zcb


[0.9913379999999999,
 0.9789249999999999,
 0.96462,
 0.941011,
 0.917136,
 0.8922580000000001,
 0.868142,
 0.845016,
 0.8218479999999999,
 0.797718]

##### Calculating c as per example 11.5 page 393. 
please note that the formula should be 2x (and not 1/2!)

In [ ]:
c = Option_IR(1,2).swap_rate(zcb)
c

0.04486177219546836